In [18]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout, Flatten, Dense, Activation, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dropout
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler

import numpy as np 
import matplotlib.pyplot as plt 
import argparse
import os
import cv2 
import random

import sys
from PIL import Image

import pickle

import math

In [11]:
DATADIR = "/Users/liaoqt/documents/CS294/data"
CATEGORIES = ["Boring", "Interesting"]
training_data = []
feature_extraction_data = []
IMG_SIZE = 300

In [12]:
## Stitch the images together 
## 0 1 2 
## 3 4 5
## 6 7 8

def stitch_images(file_path, file_name):
    images = [Image.open(image) for image in [file_path + "/" + file_name + str(x) + ".png" for x in range(100, 109)]]
    widths, heights = zip(*(i.size for i in images))
    total_width = int(sum(widths) / 3)
    total_height = int(sum(heights) / 3)
    new_image = Image.new("RGB", (total_width, total_height))
    for index in range(0, 9):
        image = images[index]
        new_image.paste(image, ((index % 3) * image.size[0], math.floor(index / 3) * image.size[1]))

    IMAGE_DIR = os.path.join(file_path, "combined/") + file_name + "combined.png"
    new_image.save(IMAGE_DIR)
    return IMAGE_DIR

In [13]:
def create_training_data():
    for category in CATEGORIES: 
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for image in sorted(os.listdir(path)): 
            if "100" not in image: ## Find the starting frame
                continue
            IMAGE_DIR = stitch_images(path, image[0:-7])
            try: 
                img_array = cv2.imread(IMAGE_DIR, cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
                if class_num == 0:
                    feature_extraction_data.append([IMAGE_DIR, False])
                else: 
                    feature_extraction_data.append([IMAGE_DIR, True])
                
            except Exception as e: 
                pass 

In [14]:
create_training_data()
random.shuffle(training_data)
random.shuffle(feature_extraction_data)

In [15]:
image_paths, labels = [], []
for image_path, label in feature_extraction_data: 
    image_paths.append(image_path)
    labels.append(label)

In [9]:
from PIL import Image 
def compute_entropy(signal):
    lensig = signal.size
    symset = list(set(signal))
    numsym = len(symset)
    propab = [np.size(signal[signal == i]) / (1.0 * lensig) for i in symset]
    entropy = np.sum([p * np.log2(1.0 / p) for p in propab])
    return entropy

In [ ]:
entropys = []
# for i in range(len(image_paths)): 
image = np.array(Image.open(image_paths[0]).convert("L"))
N = 5
S = image.shape
entropy = 0.0
for row in range(S[0]):
    for col in range(S[1]):
        Lx = max(0, col - N)
        Ux = min(S[1], col + N)
        Ly = max(0, row - N)
        Uy = min(S[0], row + N)
        region = image[Ly: Uy, Lx : Ux].flatten()
        entropy += compute_entropy(region)
entropys.append([entropy, labels[0]])

In [7]:
from skimage import data
from skimage.util import img_as_ubyte
from skimage.filters.rank import entropy
from skimage.morphology import disk

In [43]:
import torch
import pretrainedmodels
import pretrainedmodels.utils as utils
import codecs
import sys

model_name = 'nasnetalarge'
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
model.eval()

load_img = utils.LoadImage()
tf_img = utils.TransformImage(model)
features_file = open("file.csv", "ab")
feature_data = []

for i in range(len(image_paths)):
        input_img = load_img(image_paths[i])
        input_tensor = tf_img(input_img)
        input_tensor = input_tensor.unsqueeze(0)
        input = torch.autograd.Variable(input_tensor, requires_grad=False)
        output_logits = model(input)
        output_features = model.features(input)
        output_logits = model.logits(output_features)
        output_logits = output_logits[0].detach().numpy()
        row_data = np.append(output_logits, labels[i])
        feature_data = np.append(feature_data, row_data)
        
#         np.savetxt("file.csv", output_logits[0].detach().numpy())

In [54]:
feature_data = feature_data.reshape(44, 1001)

In [2]:
len(feature_data)

NameError: name 'feature_data' is not defined

In [57]:
np.savetxt("file.csv", feature_data)

In [ ]:
from image_features import image_features
n = len(image_paths)
X_train = image_features(image_paths[:n], progress=True)
y_train = no_layer[:n]
print(X_train)

In [16]:
X, y = [], []
for features, label in training_data: 
    X.append(features)
    y.append(label)
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)

In [20]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
X = X.astype('float32')
X = X / 255.0
# Split X, y into training and test data

data_sizes = [1200, 1500, 1800, 2100]
for size in data_sizes: 
    X_train, X_test, y_train, y_test = train_test_split(X[:size], y[:size], test_size = 0.1, random_state = 42) 
    model = keras.Sequential()
    model.add(Conv2D(64, (5, 5), input_shape=X.shape[1:]))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.15))

    model.add(Conv2D(64, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.15))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation("relu"))
    model.add(Dropout(0.15))

    model.add(Dense(10))
    model.add(Activation("relu"))
    model.add(Dropout(0.15))

    model.add(Dense(1))
    model.add(Activation("sigmoid"))

    model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

    with tf.device('/device:GPU:0'):
      model.fit(X_train, y_train, batch_size=2, epochs=20, verbose = 1) 

    # Do an accuracy test on data with only ~10% interesting claseses
    print('Accuracy on test---')
    y_pred_test=model.predict_classes(X_test)
    print(accuracy_score(y_test,y_pred_test))
    print(recall_score(y_test,y_pred_test))

Epoch 1/20
20/20 [==============================] - 9s 457ms/step - loss: 1.0640 - accuracy: 0.5128
Epoch 2/20
20/20 [==============================] - 9s 451ms/step - loss: 0.6918 - accuracy: 0.5641
Epoch 3/20
20/20 [==============================] - 10s 490ms/step - loss: 0.6902 - accuracy: 0.5641
Epoch 4/20
20/20 [==============================] - 9s 448ms/step - loss: 0.6907 - accuracy: 0.5641
Epoch 5/20
20/20 [==============================] - 9s 452ms/step - loss: 0.6891 - accuracy: 0.5641
Epoch 6/20
20/20 [==============================] - 9s 456ms/step - loss: 0.6912 - accuracy: 0.5641
Epoch 7/20
20/20 [==============================] - 9s 458ms/step - loss: 0.6868 - accuracy: 0.5641
Epoch 8/20
20/20 [==============================] - 9s 446ms/step - loss: 0.6843 - accuracy: 0.5641
Epoch 9/20
20/20 [==============================] - 9s 460ms/step - loss: 0.6846 - accuracy: 0.5641
Epoch 10/20
20/20 [==============================] - 9s 451ms/step - loss: 0.6891 - accuracy: 0.564

20/20 [==============================] - 9s 454ms/step - loss: 0.6865 - accuracy: 0.5641
Epoch 17/20
20/20 [==============================] - 9s 458ms/step - loss: 0.6873 - accuracy: 0.5641
Epoch 18/20
20/20 [==============================] - 9s 466ms/step - loss: 0.6885 - accuracy: 0.5641
Epoch 19/20
20/20 [==============================] - 9s 454ms/step - loss: 0.6891 - accuracy: 0.5641
Epoch 20/20
20/20 [==============================] - 9s 459ms/step - loss: 0.6858 - accuracy: 0.5641
Accuracy on test---
0.4
0.0


In [21]:
print(X_train)

[[[[1.5378702e-05]
   [1.5378702e-05]
   [1.5378702e-05]
   ...
   [1.5378702e-05]
   [1.5378702e-05]
   [1.5378702e-05]]

  [[1.5378702e-05]
   [1.5378702e-05]
   [1.5378702e-05]
   ...
   [1.5378702e-05]
   [1.5378702e-05]
   [1.5378702e-05]]

  [[1.5378702e-05]
   [1.5378702e-05]
   [1.5378702e-05]
   ...
   [1.5378702e-05]
   [1.5378702e-05]
   [1.5378702e-05]]

  ...

  [[1.5378702e-05]
   [1.5378702e-05]
   [1.5378702e-05]
   ...
   [1.5378702e-05]
   [1.5378702e-05]
   [1.5378702e-05]]

  [[1.5378702e-05]
   [1.5378702e-05]
   [1.5378702e-05]
   ...
   [1.5378702e-05]
   [1.5378702e-05]
   [1.5378702e-05]]

  [[1.5378702e-05]
   [1.5378702e-05]
   [1.5378702e-05]
   ...
   [1.5378702e-05]
   [1.5378702e-05]
   [1.5378702e-05]]]


 [[[1.5378702e-05]
   [1.5378702e-05]
   [1.5378702e-05]
   ...
   [1.5378702e-05]
   [1.5378702e-05]
   [1.5378702e-05]]

  [[1.5378702e-05]
   [1.5378702e-05]
   [1.5378702e-05]
   ...
   [1.5378702e-05]
   [1.5378702e-05]
   [1.5378702e-05]]

  [[1.5

In [22]:
len(X_train)

39

In [23]:
print(X.shape)

(44, 300, 300, 1)


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
model.fit(X, y, batch_size=16, epochs=10) 